# 2020 Befolkning i Oslo på grunnkrets

In [ ]:
import json
import os
import pandas as pd
import requests
import numpy as np
import ipyleaflet

from branca.colormap import linear
from pyjstat import pyjstat

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

Vi heter opp en GeoJSON-fil med polygoner for alle grunnkretsene i Oslo

In [ ]:
geodata = json.load(open('data/oslo-grunnkrets.json'))

Vi henter opp spørringen mot Statistikkbanken. 

In [ ]:
query = json.load(open('data/befolkning-query.json'))

Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.

In [ ]:
res = requests.post("https://data.ssb.no/api/v0/no/table/04317/", json=query)

In [ ]:
ds = pyjstat.Dataset.read(res.text)
befolkning = ds.write('dataframe', naming='id')

Vi fjerner "NaN" fra datasettet og erstatter med "0". Dette er ikke helt riktig, siden disse er fjernet av konfedensialitetshensyn. I tillegg liker ikke Cloropleth-rutinen verdier som ikke er numeriske. Vi printer også ut datastrukturen slik at vi vet hvordan data ser ut.

In [ ]:
befolkning.replace(np.NaN, 0, inplace=True)
befolkning

Vi bygger opp en mapping mellom grunnkrets-id'ene og verdiene. Disse brukes av Chropleth-rutinen som lager fargekartet. Her går det også an å velge andre basemaps om det er ønskelig. En utfordring med iPyleaflet sin Choropleth-rutine er at den forutsetter at ID-en som verdien skal mappes mot, ligger på feature-nivå i GeoJSON-fila. Dette er ikke alltid tilfellet og krever litt jobbing med kartgrunnlaget.

In [ ]:
be = dict(zip(befolkning['Grunnkretser'].tolist(), befolkning['value'].tolist()))


In [ ]:
layer = ipyleaflet.Choropleth(
    geo_data=geodata,
    choro_data=be,
    colormap=linear.YlOrRd_04,
    style={'fillOpacity': 0.6, 'dashArray': '1, 1'}
)

Vi velger et senter-punkt + zoom.

In [ ]:
center = [59.905695, 10.737880]
zoom = 12

In [ ]:
m = Map(center=center, zoom=zoom)

In [ ]:
m.add_layer(layer)
m